In [1]:
import torch
from meu_dataset import MeuDataset,avaliar_descritor,calcular_matching
from teste_util import *

PS=32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/img_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [2]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 100
from torch.utils.data import random_split, DataLoader

train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.3,0.5,0.2])
# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=True)

In [3]:
descriptor = kornia.feature.SIFTDescriptor(PS, rootsift=True).to(device).eval()

with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, descriptor,th=0.8)
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Positivos: {total_acertos} Negativos: {total_erros}')


100%|██████████| 12/12 [00:00<00:00, 32.49it/s]

Total de elementos no DataLoader: 4800
Positivos: 1580 Negativos: 55


In [4]:
from torch import nn
from external.hardnet_pytorch import HardNet 

class Descriptor(nn.Module):
        def __init__(self) -> None:
           super().__init__()           
           hardnet = HardNet()
           checkpoint = torch.load('trained_models/pretrained_nets/HardNet++.pth')
           hardnet.load_state_dict(checkpoint['state_dict'])
           self.model = hardnet
        def forward(self, x):
           return self.model(x)
    
descriptor = Descriptor().to(device).eval()


total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, descriptor,th=0.8)
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Positivos: {total_acertos} Negativos: {total_erros}')

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 93.81it/s]

Total de elementos no DataLoader: 4800
Positivos: 1769 Negativos: 17


In [5]:
import matplotlib.pyplot as plt
from tqdm import tqdm
def plot_patches_side_by_side(imgs_patches):
    num_imgs = imgs_patches.shape[0]  # Número de imagens
    fig, axs = plt.subplots(1, num_imgs, figsize=(num_imgs*4, 4))

    axs = axs.reshape((1, num_imgs))  # Ajustar a forma para matriz 2D com uma única linha

    for i in range(num_imgs):
        axs[0, i].imshow(kornia.tensor_to_image(imgs_patches[i]))
        axs[0, i].axis('off')

    plt.show()
    
# progress_bar = tqdm(dataloader_test)
# for idx, data in enumerate(progress_bar):
#     # extrair as features e orientações
#     batch_in, batch_out = data[0].to(device), data[1].to(device)
#     print(batch_in.shape)
#     plot_patches_side_by_side(batch_in[:20,0])
#     plot_patches_side_by_side(batch_out[:20,0])
    